Setup Environment

In [1]:
pip install tensorflow numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


Import Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


Load and Preprocess Dataset

In [4]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_anime_data(data_path, image_size=(64, 64)):
    images = []
    for img_file in os.listdir(data_path):
        img = load_img(os.path.join(data_path, img_file), target_size=image_size)
        img = img_to_array(img) / 127.5 - 1.0  # Normalize to [-1, 1]
        images.append(img)
    return np.array(images)

data_path = "C:/Users/User/Documents/DOKUMEN/5TIMA/MBKM/Creating anime characters using Deep Convolutional Generative Adversarial Networks (DCGANs) and Keras/images"
anime_data = load_anime_data(data_path)
print(f"Loaded {len(anime_data)} images.")


Loaded 0 images.


Build the DCGAN Model

In [6]:
def build_generator(noise_dim):
    model = tf.keras.Sequential([
        layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(noise_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        layers.Reshape((8, 8, 256)),
        
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh'),
    ])
    return model

def build_discriminator(image_shape):
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=image_shape),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Flatten(),
        layers.Dense(1),
    ])
    return model


Compile the DCGAN

In [7]:
def build_gan(generator, discriminator):
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')
    discriminator.trainable = False

    noise_dim = generator.input_shape[1]
    gan_input = layers.Input(shape=(noise_dim,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)

    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')
    return gan


Training the DCGAN

In [8]:
def train_dcgan(generator, discriminator, gan, dataset, noise_dim, epochs, batch_size):
    for epoch in range(epochs):
        for i in range(len(dataset) // batch_size):
            # Train discriminator
            real_images = dataset[i * batch_size:(i + 1) * batch_size]
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            fake_images = generator.predict(noise)

            real_labels = np.ones((batch_size, 1)) * 0.9  # Label smoothing
            fake_labels = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            misleading_labels = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, misleading_labels)

        print(f"Epoch {epoch + 1}/{epochs} | D Loss: {d_loss_real + d_loss_fake:.4f} | G Loss: {g_loss:.4f}")

        if (epoch + 1) % 10 == 0:
            generate_and_save_images(generator, epoch + 1, noise_dim)

def generate_and_save_images(generator, epoch, noise_dim):
    noise = np.random.normal(0, 1, (16, noise_dim))
    generated_images = generator.predict(noise)

    plt.figure(figsize=(4, 4))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow((generated_images[i] + 1) / 2)  # Denormalize to [0, 1]
        plt.axis('off')
    plt.savefig(f"images/epoch_{epoch}.png")
    plt.show()


Run the Model

In [9]:
noise_dim = 100
batch_size = 64
epochs = 100

generator = build_generator(noise_dim)
discriminator = build_discriminator((64, 64, 3))
gan = build_gan(generator, discriminator)

train_dcgan(generator, discriminator, gan, anime_data, noise_dim, epochs, batch_size)


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


UnboundLocalError: local variable 'd_loss_real' referenced before assignment